In [1]:
# Cell 1: YOLOv8 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Cell 2: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 2: zip 압축 해제 + split 고정 + 디렉토리 구성
import os, zipfile, shutil, random
from glob import glob
import yaml

# 경로 설정
dataset_zip_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset"
root_dir = "/content/dataset"
os.makedirs(root_dir, exist_ok=True)

# zip 해제
for fname in os.listdir(dataset_zip_dir):
    if fname.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(dataset_zip_dir, fname), 'r') as zip_ref:
            zip_ref.extractall(root_dir)

# ✅ 순서 고정된 이미지 리스트 + 라벨
image_files = sorted(glob(os.path.join(root_dir, "**", "*.jpg"), recursive=True))
label_files = [img.replace(".jpg", ".txt") for img in image_files]

# ✅ split 고정
combined = list(zip(image_files, label_files))
random.seed(42)
random.shuffle(combined)

# 8:2 split
split_idx = int(len(combined) * 0.8)
train_data = combined[:split_idx]
val_data = combined[split_idx:]

# 디렉토리 구성
for sub in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(root_dir, sub), exist_ok=True)

def copy_pairs(pairs, image_dst, label_dst):
    for img_path, lbl_path in pairs:
        if os.path.exists(img_path) and os.path.exists(lbl_path):
            shutil.copy(img_path, os.path.join(root_dir, image_dst, os.path.basename(img_path)))
            shutil.copy(lbl_path, os.path.join(root_dir, label_dst, os.path.basename(lbl_path)))

copy_pairs(train_data, "images/train", "labels/train")
copy_pairs(val_data, "images/val", "labels/val")

print(f"✅ 학습용 {len(train_data)}장, 검증용 {len(val_data)}장 복사 완료")

✅ 학습용 24000장, 검증용 6000장 복사 완료


In [ ]:
import yaml

yaml_path = os.path.join(root_dir, "data.yaml")  # /content/dataset/data.yaml

data_yaml = {
    "path": root_dir,               # "/content/dataset"
    "train": "images/train",
    "val": "images/val",
    "nc": 1,                        # 클래스 수
    "names": ["Seatbelt"]          # 클래스 이름
}

with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"✅ data.yaml 생성 완료 → {yaml_path}")

✅ data.yaml 생성 완료 → /content/dataset/data.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # 작은 모델로 빠르게 시작 (yolov8s.pt, yolov8m.pt 등도 가능)

model.train(
    data=os.path.join(root_dir, "data.yaml"),
    epochs=35,
    imgsz=800,
    batch=16,
    project='/content/drive/MyDrive/Colab Notebooks/Movon/runs',
    name='yolov8_seatbelt',
    exist_ok=True # 이전 학습 덮어씌우기
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 94.0MB/s]


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=35, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_seatbelt, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

100%|██████████| 755k/755k [00:00<00:00, 20.8MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      29/35      5.05G     0.2423     0.1424     0.8796          3        800: 100%|██████████| 1500/1500 [09:09<00:00,  2.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [00:43<00:00,  4.31it/s]

                   all       5995       5439      0.997      0.997      0.995      0.981



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/35      5.05G     0.2371     0.1381      0.878         15        800:  37%|███▋      | 557/1500 [03:27<08:47,  1.79it/s]

In [ ]:
### 이어서 학습 재개

In [ ]:
# Cell 0: 패키지 설치 (필수)
!pip install --upgrade ultralytics

In [2]:
# Cell 1: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cell 2: 데이터 unzip + split 고정
import os, zipfile, shutil, random
from glob import glob
import yaml

# 설정
dataset_zip_dir = "/content/drive/MyDrive/Colab Notebooks/Movon/dataset"
root_dir = "/content/dataset"
os.makedirs(root_dir, exist_ok=True)

# zip 해제
for fname in os.listdir(dataset_zip_dir):
    if fname.endswith(".zip"):
        with zipfile.ZipFile(os.path.join(dataset_zip_dir, fname), 'r') as zip_ref:
            zip_ref.extractall(root_dir)

# ✅ 항상 같은 순서를 유지하기 위해 sorted 사용
image_files = sorted(glob(os.path.join(root_dir, "**", "*.jpg"), recursive=True))
label_files = [img.replace(".jpg", ".txt") for img in image_files]

# ✅ 동일한 split을 위해 고정 시드 + 고정 순서
combined = list(zip(image_files, label_files))
random.seed(42)
random.shuffle(combined)

# Split
split_idx = int(len(combined) * 0.8)
train_data = combined[:split_idx]
val_data = combined[split_idx:]

# YOLO 폴더 생성
for sub in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(root_dir, sub), exist_ok=True)

# 복사 함수
def copy_pairs(pairs, image_dst, label_dst):
    for img_path, lbl_path in pairs:
        if os.path.exists(img_path) and os.path.exists(lbl_path):
            shutil.copy(img_path, os.path.join(root_dir, image_dst, os.path.basename(img_path)))
            shutil.copy(lbl_path, os.path.join(root_dir, label_dst, os.path.basename(lbl_path)))

copy_pairs(train_data, "images/train", "labels/train")
copy_pairs(val_data, "images/val", "labels/val")

print(f"✅ 복구 완료: 학습용 {len(train_data)}, 검증용 {len(val_data)}")

✅ 복구 완료: 학습용 24000, 검증용 6000


In [4]:
# Cell 3: data.yaml 자동 생성
yaml_path = os.path.join(root_dir, "data.yaml")

data_yaml = {
    "path": root_dir,
    "train": "images/train",
    "val": "images/val",
    "nc": 1,
    "names": ["Seatbelt"]
}

with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print(f"✅ data.yaml 생성 완료 → {yaml_path}")

✅ data.yaml 생성 완료 → /content/dataset/data.yaml


In [5]:
# Cell 4: 이어서 학습 재개 (resume=True)
from ultralytics import YOLO

last_ckpt = "/content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/last.pt"

# 이전 학습 이어서 시작
model = YOLO(last_ckpt)

model.train(
    data=yaml_path,
    epochs=6,  ############## 추가 학습할 epoch 수
    imgsz=800,
    batch=16,
    project="/content/drive/MyDrive/Colab Notebooks/Movon/runs",
    name="yolov8_seatbelt",
    resume=True  # 💡 중단 이어서 학습
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=35, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, li

100%|██████████| 755k/755k [00:00<00:00, 107MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 355/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 289MB/s]


...

                   all       5995       5439      0.998      0.997      0.995      0.982



6 epochs completed in 1.080 hours.
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [00:44<00:00,  4.26it/s]


                   all       5995       5439      0.998      0.997      0.995      0.982
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/Movon/runs/yolov8_seatbelt


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e8f91379450>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 